In [4]:
import numpy as np
from scipy.stats import binom
from scipy.stats import truncnorm
from scipy.stats import gamma
from scipy.special import factorial

In [5]:
# Parameters 

h = 1/7
pho = 2/3
g = 2/3
R = 19
T = 7*R

In [83]:
def K(k, betas):
    K = np.zeros((4,4))
    K[0, 0], K[0, 1], K[1, 1], K[1, 2], K[2, 2], K[2, 3], K[3, 3] = np.exp(-betas[k]*h), 1-np.exp(-betas[k]*h), np.exp(-h*pho), 1-np.exp(-h*pho), np.exp(-h*g), 1-np.exp(-h*g), 1
    return(K)

In [84]:
def division(P, Q, G):
    Q_new = Q
    for s in np.argwhere(Q == 0):
        if P[s[0]][s[1]]==0:
             Q_new[s[0]][s[1]] = 1
    M = np.divide(P, Q_new)
    for s in np.argwhere(Q == 0):
        M[s[0]][s[1]] = 0
    return M

In [85]:
# Initial settings
X1 = np.array([948, 0, 1, 0])
X2 = np.array([1689, 0, 1, 0])
X3 = np.array([3466, 0, 1, 0])
X4 = np.array([1894, 0, 1, 0])

def PAL(y, theta, R=19):
    X = np.array([X1, X2, X3, X4]) 
    X_all = np.array([X for r in range(R+1)])
    B = theta[:16].reshape(4,4)
    L = np.zeros((R, 4))
    for r in range(R):
        G = np.zeros((7, 4, 4, 4))
        for i in range(6):
            I =  X[:, 2]   
            betas = 1/8000 * np.dot(B, I)
            for k in range(4):
                print(K(k, betas))
                G[i][k] = np.multiply(np.outer(np.ones(4), X[k]), K(k, betas))
                X[k] = np.transpose(np.dot(np.transpose(np.ones(4)), G[i][k]))
        I =  X[:, 2]   
        betas = 1/8000 * np.dot(B, I)
        for k in range(4):
            G[6][k] = np.multiply(np.outer(np.ones(4), X[k]), K(k, betas))
            M = np.zeros((4,4))
            Q = np.zeros((4,4))
            Q[1][2] = theta[k+16]
            for i in range(7):
                M = M + np.multiply(G[i][k], Q)
            Y_bar = np.zeros((4,4))
            Y_bar[1][2] = y[r][k]
            G_bar = np.multiply(np.outer(np.ones(4), np.ones(4))-Q, G[i][k]) + np.multiply(Y_bar, np.multiply(G[6][k], division(Q, M, G[6][k])))
            X[k] = np.transpose(np.dot(np.transpose(np.ones(4)), G_bar))
            L[r][k] = -np.dot(np.transpose(np.ones(4)), np.dot(M, np.ones(4))) + np.dot(np.transpose(np.ones(4)), np.dot(np.multiply(Y_bar, np.log(M)), np.ones(4))) - np.dot(np.transpose(np.ones(4)), np.dot(np.log(factorial(Y_bar)), np.ones(4)))
    return L

In [86]:
theta = np.concatenate((np.random.gamma(5, 1, 16), truncnorm.rvs(a=0, b=1, loc=0.5, scale=0.5, size=4)), axis=None)
Y = np.array([[0,0,1,1],[0,2,6,1],[0,2,4,2],[23,73,63,11],[63,208,173,41],[73,207,171,27],[66,150,143,7],[26,40,87,29],[17,18,33,12],[3,4,13,6],[2,6,16,5],[1,6,11,3],[0,1,6,5],[0,2,2,2],[0,1,3,0],[0,1,4,6],[0,1,3,0],[2,1,7,1], [1,1,6,2]])
PAL(Y, theta, R=19)

[[9.99782689e-01 2.17310638e-04 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 9.09156443e-01 9.08435571e-02 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 9.09156443e-01 9.08435571e-02]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 1.00000000e+00]]
[[9.99630000e-01 3.69999624e-04 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 9.09156443e-01 9.08435571e-02 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 9.09156443e-01 9.08435571e-02]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 1.00000000e+00]]
[[9.99525334e-01 4.74666423e-04 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 9.09156443e-01 9.08435571e-02 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 9.09156443e-01 9.08435571e-02]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 1.00000000e+00]]
[[9.99717853e-01 2.82147449e-04 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 9.09156443e-01 9.08435571e-02 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 9.09156443e-01 9.08435571e-02]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 1.000

/tmp/ipykernel_1037/3815799835.py:6: RuntimeWarning: divide by zero encountered in divide
  M = np.divide(P, Q_new)
/tmp/ipykernel_1037/1226790434.py:34: RuntimeWarning: divide by zero encountered in log
  L[r][k] = -np.dot(np.transpose(np.ones(4)), np.dot(M, np.ones(4))) + np.dot(np.transpose(np.ones(4)), np.dot(np.multiply(Y_bar, np.log(M)), np.ones(4))) - np.dot(np.transpose(np.ones(4)), np.dot(np.log(factorial(Y_bar)), np.ones(4)))
/tmp/ipykernel_1037/1226790434.py:34: RuntimeWarning: invalid value encountered in multiply
  L[r][k] = -np.dot(np.transpose(np.ones(4)), np.dot(M, np.ones(4))) + np.dot(np.transpose(np.ones(4)), np.dot(np.multiply(Y_bar, np.log(M)), np.ones(4))) - np.dot(np.transpose(np.ones(4)), np.dot(np.log(factorial(Y_bar)), np.ones(4)))


array([[ -15.37353026,  -15.43530625,  -16.10278382,  -16.17351937],
       [          nan,          -inf,  -25.13406636,  -16.17351937],
       [          nan,          -inf,  -20.75205645,  -17.51974451],
       [         -inf,          -inf, -247.51728863,  -40.20660677],
       [         -inf,          -inf,          -inf, -128.4322564 ],
       [         -inf,          -inf,          -inf,  -48.29415931],
       [         -inf,          -inf, -119.92008842,  -17.18302553],
       [         -inf,          -inf,  -36.89670527,  -43.62574612],
       [         -inf,          -inf,  -19.51966484,  -18.81049605],
       [         -inf,          -inf,  -32.4219698 ,  -16.87376117],
       [         -inf,          -inf,  -27.2725747 ,  -16.87364523],
       [         -inf,          -inf,  -30.90208081,  -17.28216306],
       [          nan,          -inf,  -36.18673996,  -16.74437019],
       [          nan,          -inf,  -41.76437966,  -17.28882666],
       [          nan,          -i

In [37]:
Q = np.zeros((4, 4))
print(np.argwhere(Q == 0))

[[0 0]
 [0 1]
 [0 2]
 [0 3]
 [1 0]
 [1 1]
 [1 2]
 [1 3]
 [2 0]
 [2 1]
 [2 2]
 [2 3]
 [3 0]
 [3 1]
 [3 2]
 [3 3]]
